In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
import os
os.getcwd()
os.chdir("C:\\Users\\Home\\Desktop\\Saptha\\Machine-Learning\\WNS")

In [3]:
# For cluster computing using ipyparallel.
import ipyparallel as ipp
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()
print (rc[:].apply_sync(lambda : "Hello, World"))

# Invoke garbage collector in python
import gc 

In [3]:
# Importing Datasets

trainAnalysis = pd.read_csv('train.csv')
itemDataAnalysis = pd.read_csv('item_data.csv')
viewLogAnalysis = pd.read_csv('view_log.csv')
test = pd.read_csv('test.csv')

In [4]:
# Split dateTime as date and time seperately!!
trainAnalysis['impression_date'] = trainAnalysis.impression_time.str.split().str[0]
trainAnalysis['impression_timeStamp'] = trainAnalysis.impression_time.str.split().str[1]

In [5]:
# Exploratory Data Analysis
trainAnalysis.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,impression_date,impression_timeStamp
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,2018-11-15,00:00:00
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,2018-11-15,00:01:00
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,2018-11-15,00:02:00
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,2018-11-15,00:02:00
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,2018-11-15,00:02:00


In [6]:
trainAnalysisSorted = pysqldf("SELECT * FROM trainAnalysis order by user_id ASC, impression_date ASC, impression_timeStamp ASC;")

In [7]:
trainAnalysisSorted['impressionCountPerUserPerDay'] = pysqldf("SELECT count(*) over (partition by user_id, impression_date) AS impression_count_per_day from trainAnalysisSorted t;")

In [8]:
trainAnalysisSorted['impressionCountPerUserPerDayPerApp'] = pysqldf("SELECT count(*) over (partition by user_id, impression_date, app_code) AS impressionCountPerUserPerDayPerApp from trainAnalysisSorted t;")

In [9]:
trainAnalysisSorted['impressionCountPerUserPerApp'] = pysqldf("SELECT count(*) over (partition by user_id, app_code) AS impressionCountPerUserPerApp from trainAnalysisSorted t;")

In [10]:
print (trainAnalysisSorted.shape)
# (237609, 12)
print (trainAnalysisSorted.impression_id.nunique(dropna = True))
# 237609
# So each impression is unique, i.e. impression has nothing to do with item_id

(237609, 12)
237609


In [11]:
 # Lets encode os_version with label encoder as it is ordinally categorical.
print (trainAnalysisSorted['os_version'].value_counts())
trainAnalysisSorted.replace({'os_version':{'old':'1','intermediate':'2','latest':'3'}}, inplace=True)
print (trainAnalysisSorted['os_version'].value_counts())



latest          129216
intermediate     55543
old              52850
Name: os_version, dtype: int64
3    129216
2     55543
1     52850
Name: os_version, dtype: int64


In [13]:
print (trainAnalysisSorted.is_click.value_counts())
# 0    226747
# 1     10862
# % of 0's : 95.42%
# % of 1's : 4.57%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 3 and is_4G = 1;").is_click.value_counts())
# 0    49678
# 1     2181
# % of 0's : 95.79%
# % of 1's : 4.26%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 3;").is_click.value_counts())
# 0    123834
# 1      5382
# % of 0's : 95.83%
# % of 1's : 4.16%

print (pysqldf("SELECT * from trainAnalysisSorted where is_4G = 1;").is_click.value_counts())
# 0    82009
# 1     3842
# % of 0's : 95.52%
# % of 1's : 4.47%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 2 and is_4G = 1;").is_click.value_counts())
# 0    18169
# 1      952
# % of 0's : 95.02%
# % of 1's : 4.97%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 2;").is_click.value_counts())
# 0    52668
# 1     2875
# % of 0's : 94.82%
# % of 1's : 5.17%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 1 and is_4G = 0;").is_click.value_counts())
# 0    36083
# 1    1896
# % of 0's : 95%
# % of 1's : 4.99%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 1 and is_4G = 1;").is_click.value_counts())
# 0    14162
# 1      709
# % of 0's : 95.23%
# % of 1's : 4.76%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 1;").is_click.value_counts())
# 0    50245
# 1     2605
# % of 0's : 95%
# % of 1's : 4.92%

print (pysqldf("SELECT * from trainAnalysisSorted where is_4G = 0;").is_click.value_counts())
# 0    144738
# 1      7020
# % of 0's : 95.37%
# % of 1's : 4.62%

print (pysqldf("SELECT * from trainAnalysisSorted where os_version = 3 and is_4G = 0;").is_click.value_counts())
# 0    74156
# 1     3201
# % of 0's : 95.86%
# % of 1's : 4.13%

0    226747
1     10862
Name: is_click, dtype: int64
0    49678
1     2181
Name: is_click, dtype: int64
0    123834
1      5382
Name: is_click, dtype: int64
0    82009
1     3842
Name: is_click, dtype: int64
0    18169
1      952
Name: is_click, dtype: int64
0    52668
1     2875
Name: is_click, dtype: int64
0    36083
1     1896
Name: is_click, dtype: int64
0    14162
1      709
Name: is_click, dtype: int64
0    50245
1     2605
Name: is_click, dtype: int64
0    144738
1      7020
Name: is_click, dtype: int64
0    74156
1     3201
Name: is_click, dtype: int64


In [12]:
print (viewLogAnalysis.shape)
print (itemDataAnalysis.shape)
print (trainAnalysisSorted.user_id.nunique(dropna = True))
print (viewLogAnalysis.user_id.nunique(dropna = True))
print (trainAnalysisSorted.shape)
print (trainAnalysisSorted.is_click.value_counts())


(3118622, 5)
(132761, 6)
74723
89157
(237609, 12)
0    226747
1     10862
Name: is_click, dtype: int64


In [13]:
# pysqldf("select * from viewLogAnalysis where item_id not in (select item_id from itemDataAnalysis);").shape
# (1782, 5)

In [14]:
# pysqldf("select * from viewLogAnalysis where item_id is NULL;").shape
# (0, 5)

In [15]:
# Using Left Join because, Some items in view log table are not present in item log table.
ViewLgItmDatDf = pysqldf("SELECT V.server_time, V.device_type, V.session_id, V.user_id, V.item_id, I.item_price, I.category_1, I.category_2, I.category_3, I.product_type from viewLogAnalysis V LEFT JOIN itemDataAnalysis I ON V.item_id = I.item_id;")

print (ViewLgItmDatDf.shape)
print (viewLogAnalysis.shape)
print (viewLogAnalysis.item_id.nunique())
print (itemDataAnalysis.item_id.nunique())
print (ViewLgItmDatDf.item_id.nunique())
print (ViewLgItmDatDf.columns)

del viewLogAnalysis
del itemDataAnalysis


(3118622, 10)
(3118622, 5)
126708
132761
126708
Index(['server_time', 'device_type', 'session_id', 'user_id', 'item_id',
       'item_price', 'category_1', 'category_2', 'category_3', 'product_type'],
      dtype='object')


In [17]:
# Enabling garbage collection
gc.collect() 

216

In [ ]:
# Export the data for analysis using Excel !
# ViewLgItmDatDf.to_csv('ViewLgItmDatDf.csv') 

In [18]:
ViewLgItmDatDf.tail()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
3118617,2018-12-11 23:59:00,android,135534,49480,1013,41472.0,1.0,42.0,115.0,9461.0
3118618,2018-12-11 23:59:00,android,206169,70215,44826,2073.0,13.0,78.0,287.0,5568.0
3118619,2018-12-11 23:59:00,android,831039,4925,86608,320.0,4.0,1.0,111.0,9362.0
3118620,2018-12-11 23:59:00,android,601193,16870,38284,2176.0,10.0,43.0,223.0,10083.0
3118621,2018-12-11 23:59:00,android,138217,1007,88331,1843.0,11.0,51.0,14.0,1761.0


In [ ]:
# pysqldf("select * from ViewLgItmDatDf where item_price is NULL;").shape
# (1782, 10)

In [ ]:
# pysqldf("SELECT * from ViewLgItmDatDf group by user_id,item_id;").head(50)

In [22]:
# pysqldf("SELECT * from trainAnalysisSorted order by user_id;").head(20)

In [18]:
isClickUserId = pysqldf("SELECT DISTINCT user_id from trainAnalysisSorted where is_click = 1;")

In [19]:
# Split dateTime as date and time seperately!!
ViewLgItmDatDf['server_date'] = ViewLgItmDatDf.server_time.str.split().str[0]
ViewLgItmDatDf['server_timeStamp'] = ViewLgItmDatDf.server_time.str.split().str[1]

# Sort based on user_id!!
ViewLgItmDatDfSorted = pysqldf("SELECT * from ViewLgItmDatDf order by user_id,server_time;")

# Extracting the gap in visiting Zbay website.
ViewLgItmDatDfSorted['previousDate'] = pysqldf("SELECT lag(server_date,1) over (partition by user_id order by server_date) from ViewLgItmDatDfSorted;")
ViewLgItmDatDfSorted['previousDateFinal'] = pysqldf("select CASE WHEN previousDate is NULL THEN server_date ELSE previousDate END as previousDateFinal FROM ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['previousDate']
ViewLgItmDatDfSorted['gapInVisit'] = pd.to_datetime(ViewLgItmDatDfSorted['server_date'], format='%Y-%m-%d') - pd.to_datetime(ViewLgItmDatDfSorted['previousDateFinal'], format='%Y-%m-%d')
ViewLgItmDatDfSorted.info()
ViewLgItmDatDfSorted['gapInVisit'] = ViewLgItmDatDfSorted['gapInVisit'].dt.days.astype('int16')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118622 entries, 0 to 3118621
Data columns (total 14 columns):
server_time          object
device_type          object
session_id           int64
user_id              int64
item_id              int64
item_price           float64
category_1           float64
category_2           float64
category_3           float64
product_type         float64
server_date          object
server_timeStamp     object
previousDateFinal    object
gapInVisit           timedelta64[ns]
dtypes: float64(5), int64(3), object(5), timedelta64[ns](1)
memory usage: 333.1+ MB


In [ ]:
# Analyzing user pattern of visiting the website, for users who clicked the ad. atleast once versus users who never clicked.
# clickedAtleastOnceUserPattern = pysqldf("SELECT * from ViewLgItmDatDfSorted where user_id IN (SELECT DISTINCT user_id from trainAnalysisSorted where is_click = 1);")
# notEvenOnceClickedUserPattern = pysqldf("SELECT * from ViewLgItmDatDfSorted where user_id NOT IN (SELECT DISTINCT user_id from trainAnalysisSorted where is_click = 1);")

# Export the data for analysis using Excel !
# clickedAtleastOnceUserPattern.to_csv('clickedAtleastOnceUserPattern.csv') 
# notEvenOnceClickedUserPattern.to_csv('notEvenOnceClickedUserPattern.csv') 

In [20]:
ViewLgItmDatDfSorted['noOfLogs'] = pysqldf("select count(*) over (partition by user_id order by server_time) from ViewLgItmDatDfSorted;")
log = pysqldf("select user_id, server_time, noOfLogs from ViewLgItmDatDfSorted;")
trainAnalysisSorted['NoOfLgsBfThtDay'] = pysqldf("select max(noOfLogs) from trainAnalysisSorted inner join log where trainAnalysisSorted.user_id = log.user_id and ((trainAnalysisSorted.impression_time > log.server_time) or (trainAnalysisSorted.impression_time = log.server_time))  group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
del log
#trainAnalysisSorted['NoOfLgsBfThtDay'].isna().sum()
trainAnalysisSorted['NoOfLgsBfThtDay'].fillna(0,inplace=True)

In [21]:
# ViewLgItmDatDfSorted['device_type'].value_counts()
device = pysqldf("select user_id, device_type, count(*) from ViewLgItmDatDfSorted group by user_id,device_type;")
trainAnalysisSorted['device_type'] = pysqldf("select d.device_type from trainAnalysisSorted t inner join device d where t.user_id = d.user_id;")


In [22]:
# device_type is not ordinal data, so lets dummy encode it.
trainAnalysisSorted = pd.get_dummies(trainAnalysisSorted, columns=['device_type'], prefix = ['device'])
del device

In [23]:
# Determining how to choose bins for price tags!!!
print (np.max(ViewLgItmDatDfSorted['item_price']))
print (np.min(ViewLgItmDatDfSorted['item_price']))
print (pd.cut(ViewLgItmDatDfSorted['item_price'], [0,500,1000,5000,10000,25000,50000,100000,150000,200000,250000,500000,750000,1000000,1250000,1500000]).value_counts())
ViewLgItmDatDfSorted['item_price_bins'] = pd.cut(ViewLgItmDatDfSorted['item_price'], [0,500,1000,5000,10000,25000,50000,100000,150000,200000,250000,500000,750000,1000000,1250000,1500000],labels=["[0, 500]", "[500, 1000]", "[1000, 5000]", "[5000, 10000]", "[10000, 25000]","[25000, 50000]","[50000, 100000]","[100000,150000]","[150000,200000]","[200000,250000]","[250000,500000]","[500000, 750000]", "[750000, 1000000]", "[1000000, 1250000]", "[1250000, 1500000]"])

1340800.0
5.0
(1000, 5000]          1458012
(5000, 10000]          416693
(10000, 25000]         398256
(500, 1000]            355306
(0, 500]               190783
(25000, 50000]         147193
(50000, 100000]        100404
(100000, 150000]        26943
(150000, 200000]        16339
(200000, 250000]         4588
(250000, 500000]         2104
(500000, 750000]          192
(750000, 1000000]          26
(1250000, 1500000]          1
(1000000, 1250000]          0
Name: item_price, dtype: int64


In [24]:
# Enabling garbage collection
gc.collect() 

339

## Determining how to choose bins for price tags!!!
print (np.max(ViewLgItmDatDfSorted['item_price']))
print (np.min(ViewLgItmDatDfSorted['item_price']))
print (pd.cut(ViewLgItmDatDfSorted['item_price'], [0,500,1000,5000,10000,25000,50000,100000,150000,200000,250000,500000,750000,1000000,1250000,1500000]).value_counts())
ViewLgItmDatDfSorted['item_price_bins'] = pd.cut(ViewLgItmDatDfSorted['item_price'], [0,500,1000,5000,10000,25000,50000,100000,150000,200000,250000,500000,750000,1000000,1250000,1500000])

In [25]:
ViewLgItmDatDfSorted['_0_500_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[0, 500]', '_0_500_YN'] = 1 
ViewLgItmDatDfSorted['_0_500_count'] = pysqldf("SELECT SUM(_0_500_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_0_500_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_500_1000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[500, 1000]', '_500_1000_YN'] = 1 
ViewLgItmDatDfSorted['_500_1000_count'] = pysqldf("SELECT SUM(_500_1000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_500_1000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_1000_5000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[1000, 5000]', '_1000_5000_YN'] = 1  
ViewLgItmDatDfSorted['_1000_5000_count'] = pysqldf("SELECT SUM(_1000_5000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_1000_5000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_5000_10000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[5000, 10000]', '_5000_10000_YN'] = 1 
ViewLgItmDatDfSorted['_5000_10000_count'] = pysqldf("SELECT SUM(_5000_10000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_5000_10000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_10000_25000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[10000, 25000]', '_10000_25000_YN'] = 1 
ViewLgItmDatDfSorted['_10000_25000_count'] = pysqldf("SELECT SUM(_10000_25000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_10000_25000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_25000_50000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[25000, 50000]', '_25000_50000_YN'] = 1  
ViewLgItmDatDfSorted['_25000_50000_count'] = pysqldf("SELECT SUM(_25000_50000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_25000_50000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_50000_100000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[50000, 100000]', '_50000_100000_YN'] = 1  
ViewLgItmDatDfSorted['_50000_100000_count'] = pysqldf("SELECT SUM(_50000_100000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_50000_100000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_100000_150000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[100000,150000]', '_100000_150000_YN'] = 1  
ViewLgItmDatDfSorted['_100000_150000_count'] = pysqldf("SELECT SUM(_100000_150000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_100000_150000_YN'] 
print (" ****** Executed ****** ")
 
ViewLgItmDatDfSorted['_150000_200000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[150000,200000]', '_150000_200000_YN'] = 1  
ViewLgItmDatDfSorted['_150000_200000_count'] = pysqldf("SELECT SUM(_150000_200000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_150000_200000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_200000_250000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[200000,250000]', '_200000_250000_YN'] = 1  
ViewLgItmDatDfSorted['_200000_250000_count'] = pysqldf("SELECT SUM(_200000_250000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_200000_250000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_250000_500000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[250000,500000]', '_250000_500000_YN'] = 1  
ViewLgItmDatDfSorted['_250000_500000_count'] = pysqldf("SELECT SUM(_250000_500000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_250000_500000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_500000_750000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[500000, 750000]', '_500000_750000_YN'] = 1
ViewLgItmDatDfSorted['_500000_750000_count'] = pysqldf("SELECT SUM(_500000_750000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_500000_750000_YN']
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_750000_1000000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[750000, 1000000]', '_750000_1000000_YN'] = 1 
ViewLgItmDatDfSorted['_750000_1000000_count'] = pysqldf("SELECT SUM(_750000_1000000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_750000_1000000_YN'] 
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_1000000_1250000_YN'] = 0


ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[1000000, 1250000]', '_1000000_1250000_YN'] = 1 
ViewLgItmDatDfSorted['_1000000_1250000_count'] = pysqldf("SELECT SUM(_1000000_1250000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_1000000_1250000_YN']
gc.collect() 
print (" ****** Executed ****** ")

ViewLgItmDatDfSorted['_1250000_1500000_YN'] = 0
ViewLgItmDatDfSorted.loc[ViewLgItmDatDfSorted.item_price_bins == '[1250000, 1500000]', '_1250000_1500000_YN'] = 1
ViewLgItmDatDfSorted['_1250000_1500000_count'] = pysqldf("SELECT SUM(_1250000_1500000_YN) over (PARTITION BY user_id order by server_time) from ViewLgItmDatDfSorted;")
del ViewLgItmDatDfSorted['_1250000_1500000_YN']
gc.collect() 
print (" ****** Executed ****** ")


 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 


In [4]:
# Making sure that dates are interpreted properly, before joining both the tables.
format = '%Y-%m-%d %H:%M'
ViewLgItmDatDfSorted['server_time'] = pd.to_datetime(ViewLgItmDatDfSorted['server_time'], format=format)
format = '%Y-%m-%d %H:%M'
trainAnalysisSorted['impression_time'] = pd.to_datetime(trainAnalysisSorted['impression_time'], format=format)

In [5]:
trainAnalysisSorted['_0_500_count'] = pysqldf("select max(_0_500_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_500_1000_count'] = pysqldf("select max(_500_1000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_1000_5000_count'] = pysqldf("select max(_1000_5000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_5000_10000_count'] = pysqldf("select max(_5000_10000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_10000_25000_count'] = pysqldf("select max(_10000_25000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_25000_50000_count'] = pysqldf("select max(_25000_50000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_50000_100000_count'] = pysqldf("select max(_50000_100000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_100000_150000_count'] = pysqldf("select max(_100000_150000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_150000_200000_count'] = pysqldf("select max(_150000_200000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_200000_250000_count'] = pysqldf("select max(_200000_250000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_250000_500000_count'] = pysqldf("select max(_250000_500000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_500000_750000_count'] = pysqldf("select max(_500000_750000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_750000_1000000_count'] = pysqldf("select max(_750000_1000000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_1000000_1250000_count'] = pysqldf("select max(_1000000_1250000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")
trainAnalysisSorted['_1250000_1500000_count'] = pysqldf("select max(_1250000_1500000_count) from trainAnalysisSorted inner join ViewLgItmDatDfSorted where trainAnalysisSorted.user_id = ViewLgItmDatDfSorted.user_id and ((trainAnalysisSorted.impression_time > ViewLgItmDatDfSorted.server_time) or (trainAnalysisSorted.impression_time = ViewLgItmDatDfSorted.server_time)) group by trainAnalysisSorted.user_id,trainAnalysisSorted.impression_time;")
print (" ****** Executed ****** ")


 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 
 ****** Executed ****** 


In [6]:
# Determine the time of day, day of the week, day of month the impression was shown to the user.
import datetime

# The day of the week with Monday=0, Sunday=6.
trainAnalysisSorted['day_of_week'] = pd.to_datetime(trainAnalysisSorted['impression_date'], format  = '%Y-%m-%d').dt.dayofweek

trainAnalysisSorted['day_of_month'] = pd.to_datetime(trainAnalysisSorted['impression_date'], format = '%Y-%m-%d').dt.day

trainAnalysisSorted['time_of_day'] = pd.to_datetime(trainAnalysisSorted['impression_timeStamp'], format = '%H:%M:%S').dt.hour

